# Meteo con el pato

### Este informe muestra los datos diarios de temperatura recogidos en el portal de datos abiertos del ayuntamiento.
* dataset: **meteorologia-tiempo-real** 

In [1]:
%%capture
import duckdb
import hvplot.duckdb
import panel as pn
import hvplot.duckdb


In [2]:


%load_ext sql
#conexion a bbdd en memoria o en fichero
conn = duckdb.connect() 
#conn = duckdb.connect('/home/jcaubin/duck_test.db') 

%sql conn --alias duckdb

In [3]:
%%capture
%%sql 

create or replace table meteo_today as
select * from read_csv('https://datos.madrid.es/egob/catalogo/300392-11041819-meteorologia-tiempo-real.csv');

In [4]:
%%capture
%%sql

create or replace table estaciones as
select * from read_csv('https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv')

In [5]:
%%capture
%%sql

create or replace table magnitudes as
select cast(CODIGO as int) CODIGO, PARAMETRO, UNIDAD, cast(COD_TECNICA as int) COD_TECNICA, TECNICA 
from read_csv('../maestras/parametros_meteo.csv')

In [6]:
%%capture
%%sql

create or replace table td as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALOR	 from (
from meteo_today
unpivot (
    (VALOR)
for HORA in (
    H01, 
    H02, 
    H03, 
    H04, 
    H05, 
    H06, 
    H07, 
    H08, 
    H09, 
    H10, 
    H11, 
    H12, 
    H13, 
    H14, 
    H15, 
    H16, 
    H17, 
    H18, 
    H19, 
    H20, 
    H21, 
    H22, 
    H23, 
    H24 
    ) 
)
);

In [7]:
%%capture
%%sql 

create or replace  table tv as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALIDEZ	 
from (
from meteo_today
unpivot (
    (VALIDEZ)
for HORA in (
    V01, 
    V02, 
    V03, 
    V04, 
    V05, 
    V06, 
    V07, 
    V08, 
    V09, 
    V10, 
    V11, 
    V12, 
    V13, 
    V14, 
    V15, 
    V16, 
    V17, 
    V18, 
    V19, 
    V20, 
    V21, 
    V22, 
    V23, 
    V24, 
    ) 
 ))
 ;


In [8]:
%%capture
%%sql

create or replace table meteo_madrid24 as
select td.*, tv.VALIDEZ, e.ESTACION ESTACION_DESC, e.ALTITUD, m.PARAMETRO
from td 
inner join tv on td.PROVINCIA=tv.PROVINCIA and 	td.MUNICIPIO=tv.MUNICIPIO and td.ESTACION=tv.ESTACION and 	td.MAGNITUD = tv.MAGNITUD
    and 	td.PUNTO_MUESTREO = tv.PUNTO_MUESTREO and td.ANO= tv.ANO and td.MES = tv.MES and TD.DIA = tv.DIA and td.H=tv.H
inner join estaciones e on td.estacion = e.CÓDIGO_CORTO
inner join magnitudes m on m.CODIGO = tv.MAGNITUD    

## Temperatura

In [9]:
query  = conn.sql("""
select estacion_desc, count(1)
from  meteo_madrid24
where
magnitud = 83 --temperatura
and validez = 'V'
group by all;
""")

estaciones = query.fetchall()

estaciones_list = sorted([e[0] for e in estaciones])



In [ ]:
relation = conn.sql(
    """PIVOT(
select estacion_desc, h, valor
from meteo_madrid24
where
magnitud = 83 --temperatura
and validez = 'V'
)
on ESTACION_DESC
USING sum(VALOR)
group by h"""
)
el = ["El Pardo", "Escuelas Aguirre", "Casa de Campo", "Plaza España", 'Peñagrande']

plot = relation.hvplot.line(
    y=el, x="h"
)

pn.extension()  # for notebook
pn.Column(plot)

## Presion

In [ ]:
relation2 = conn.sql(
    """PIVOT(
select estacion_desc, h, valor
from meteo_madrid24
where
magnitud = 87
and validez = 'V'
)
on ESTACION_DESC
USING sum(VALOR)
group by h"""
)

plot2 = relation2.hvplot.line(
    y=["Casa de Campo", "Juan Carlos I"], x="H"
)

pn.extension()  # for notebook
pn.Column(plot2)

In [ ]:
# imprime la hora de ejecución
from datetime import datetime
from IPython.display import display, Markdown

t = datetime.now()
display(Markdown(f"### hora de ejecución: {t}"))